In [1]:
# ## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

# copyfile(src="../input/input2/train_unlabeld_dataloader.p", dst="../working/train_unlabelled_dataloader.p")
# copyfile(src="../input/input2/train_labeled_dataloader.p", dst="../working/train_labelled_dataloader.p")

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/home/jubly/FairFrame/evaluation.py'>

## Get Dataloaders

In [3]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [4]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [5]:
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [6]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [7]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [8]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [9]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [10]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [11]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [12]:
## LOCAL - 2nd line // KAGGLE -- 1st line
# glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [13]:
embedding_index = load_embeddings(glove_twitter)

In [14]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [15]:
len(review_dict.tokens)

16256

In [16]:
len(unknown_words)

4428

In [17]:
# for word in unknown_words:
#     print(word)

## Neural Network LSTM Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [ ]:
def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False
        
def unfreeze_model(model):
    for param in model.parameters():
        param.requires_grad = True

In [11]:
class LSTM_model(nn.Module):
    """
    LSTM classification model using pretrained glove embeddings
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, embedding_matrix, num_hidden_layers = 3, hidden_size = 100, num_classes = 2):
        super(LSTM_model, self).__init__()
        vocab_size = embedding_matrix.shape[0]
        embed_size = embedding_matrix.shape[1]
        self.embedding_matrix = embedding_matrix
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=0)    
        self.embed.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        #self.embed.weight.requires_grad = unfrozen
        
        self.lstm = nn.LSTM(self.embed_size,self.hidden_size,self.num_hidden_layers, batch_first=True,bidirectional= True,bias=True)
        
        self.projection = nn.Linear(2*self.hidden_size, self.num_classes, bias=True)

    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        lstm_output = self.lstm(embedding)
        # lstm_output is a tuple containing lstm output and (hidden_state, lstm_cell). 
        # lstm_output[0] would be of shape "batch_size x num_tokens x hidden_size" (VERIFY)
        
        logits = self.projection(lstm_output[0])
        # logits would be of shape "batch_size x num_tokens x num_classes (2)" (VERIFY)
        
        batch_size, _, __ = logits.shape
        
        #selecting the logit at the flagged index
        relevant_logits = logits[list(range(batch_size)),flagged_index]
        # relevant_logits would be of shape "batch_size x num_classes (2)" (VERIFY)
        
        return relevant_logits

## First performing fully supervised learning using the labelled set to train new vector representations

In [19]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = LSTM_model(glove_embedding_index, num_hidden_layers = 3, hidden_size = 100, num_classes = 2).to(current_device)

In [20]:
criterion = nn.CrossEntropyLoss(reduction='sum')
#optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

## Supervised model training

In [21]:
def train_supervised_model(model, criterion, train_loader_labelled, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    num_first_epochs = 10
    num_second_epochs = num_epochs - num_first_epohcs
    
    # freeze part
    freeze_model(model)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_first_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        total_epoch_loss = 0
        
        for i,(tokens_labelled, labels, flagged_indices_labelled) in tqdm(enumerate(train_loader_labelled)):
            
            tokens_labelled = tokens_labelled.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            labels = labels.to(current_device)

            # forward pass and compute loss
            logits = model(tokens_labelled,flagged_indices_labelled)
            
            loss = criterion(logits, labels)
        
            # run update step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += loss

            if i % print_every == 0:
                losses = loss/len(tokens_labelled)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader_labelled.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            logits = model(tokens,flagged_indices)
            
            loss = criterion(logits, labels)
            
            #Add loss to the validation loss
            total_validation_loss += loss

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_labelled.pt')
            torch.save(train_losses, path_to_save+'train_losses_labelled')
            torch.save(val_losses, path_to_save+'val_losses_labelled')
            torch.save(opts, path_to_save+'opts_labelled')
            
    # unfreeze part
    unfreeze_model(model)
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_second_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        total_epoch_loss = 0
        
        for i,(tokens_labelled, labels, flagged_indices_labelled) in tqdm(enumerate(train_loader_labelled)):
            
            tokens_labelled = tokens_labelled.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            labels = labels.to(current_device)

            # forward pass and compute loss
            logits = model(tokens_labelled,flagged_indices_labelled)
            
            loss = criterion(logits, labels)
        
            # run update step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += loss

            if i % print_every == 0:
                losses = loss/len(tokens_labelled)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader_labelled.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            logits = model(tokens,flagged_indices)
            
            loss = criterion(logits, labels)
            
            #Add loss to the validation loss
            total_validation_loss += loss

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_labelled.pt')
            torch.save(train_losses, path_to_save+'train_losses_labelled')
            torch.save(val_losses, path_to_save+'val_losses_labelled')
            torch.save(opts, path_to_save+'opts_labelled')
        
    return model, train_losses, val_losses

In [22]:
path = os.getcwd()
model_folder = 'lstm_model/'
model_dir = path + '/models/' + model_folder

In [23]:
len(train_loader_labelled)

114

In [24]:
lstm_model, train_losses, val_losses = train_supervised_model(model, criterion, train_loader_labelled, val_loader, num_epochs=15, path_to_save=model_dir)


2019-11-24 17:41:33.027204 | Epoch 0


Average training loss at batch  0 : 0.735

Average training loss after epoch  0 : 0.291
Average validation loss after epoch  0 : 0.252
2019-11-24 17:41:46.616863 | Epoch 1


Average training loss at batch  0 : 0.230

Average training loss after epoch  1 : 0.220
Average validation loss after epoch  1 : 0.346
2019-11-24 17:42:00.425754 | Epoch 2


Average training loss at batch  0 : 0.437

Average training loss after epoch  2 : 0.203
Average validation loss after epoch  2 : 0.232
2019-11-24 17:42:14.009057 | Epoch 3


Average training loss at batch  0 : 0.283

Average training loss after epoch  3 : 0.179
Average validation loss after epoch  3 : 0.291
2019-11-24 17:42:28.000180 | Epoch 4


...

Average training loss at batch  0 : 0.235

Average training loss after epoch  5 : 0.143
Average validation loss after epoch  5 : 0.266
2019-11-24 17:42:55.238721 | Epoch 6


Average training loss at batch  0 : 0.075

Average training loss after epoch  6 : 0.124
Average validation loss after epoch  6 : 0.322
2019-11-24 17:43:08.867896 | Epoch 7


Average training loss at batch  0 : 0.045

Average training loss after epoch  7 : 0.098
Average validation loss after epoch  7 : 0.260
2019-11-24 17:43:22.675940 | Epoch 8


Average training loss at batch  0 : 0.046

Average training loss after epoch  8 : 0.087
Average validation loss after epoch  8 : 0.392
2019-11-24 17:43:36.501053 | Epoch 9


Average training loss at batch  0 : 0.027

Average training loss after epoch  9 : 0.083
Average validation loss after epoch  9 : 0.359


### Clustering Stuff

In [25]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = self.lmbda * cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [26]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)

## Dataloader stuff

In [27]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)

    return tokens, labels, flagged_indices, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)

    return tokens, labels, flagged_indices, train_loader_unlabelled_iter

### Training Function

In [28]:
def train_clusters(model, centroids, criterion, train_loader_labelled, train_loader_unlabelled, valid_loader, num_epochs=15 num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    num_first_epochs = 10
    num_second_epochs = num_epochs - num_first_epohcs
    
    # freeze part
    freeze_model(model)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_first_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.eval() # we're only clustering, not training model
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in tqdm(range(int(num_batches))):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_labelled.data + cluster_loss_unlabelled.data
            
#             #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss.data

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_labelled.detach(), sentence_embed_labelled.detach())
    
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_unlabelled.detach(), sentence_embed_unlabelled.detach())

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_unlabelled.pt')
            torch.save(centroids, path_to_save+'centroids_unlabelled')
            torch.save(train_losses, path_to_save+'train_losses_unlabelled')
            torch.save(val_losses, path_to_save+'val_losses_unlabelled')
            torch.save(opts, path_to_save+'opts_unlabelled')
    
    
    # unfreeze part
    unfreeze_model(model)
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
    
    optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)
    
    for epoch in range(num_second_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.eval() # we're only clustering, not training model
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in tqdm(range(int(num_batches))):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_labelled.data + cluster_loss_unlabelled.data
            
#             #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss.data

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_labelled.detach(), sentence_embed_labelled.detach())
    
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_unlabelled.detach(), sentence_embed_unlabelled.detach())

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_unlabelled.pt')
            torch.save(centroids, path_to_save+'centroids_unlabelled')
            torch.save(train_losses, path_to_save+'train_losses_unlabelled')
            torch.save(val_losses, path_to_save+'val_losses_unlabelled')
            torch.save(opts, path_to_save+'opts_unlabelled')
        
    return model, centroids, train_losses, val_losses

In [29]:
unsupervised_model = model
unsupervised_model.projection = nn.Identity()

In [30]:
centroids = centroid_init(2, 2*unsupervised_model.hidden_size, train_loader_labelled, unsupervised_model, current_device)
criterion = KMeansCriterion(1).to(current_device)
#optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [31]:
centroids.shape

torch.Size([2, 200])

In [32]:
path = os.getcwd()
model_folder = 'lstm_unfrozen_model/'
model_dir = path + '/models/' + model_folder

In [33]:
num_batches = int(len(train_loader_unlabelled.dataset)/train_loader_unlabelled.batch_size)+1
num_batches

3211

In [34]:
lstm_model, lstm_centroids, lstm_train_losses, lstm_val_losses = train_clusters(unsupervised_model, centroids, criterion, train_loader_labelled,train_loader_unlabelled, val_loader, num_epochs=15, num_batches=num_batches, path_to_save=model_dir)


2019-11-24 17:44:01.365323 | Epoch 0


Average training loss at batch  0 : 1.211
Average training loss at batch  1000 : 1.229
Average training loss at batch  2000 : 1.669
Average training loss at batch  3000 : 0.935

Average training loss after epoch  0 : 2.681
Average validation loss after epoch  0 : 1.249
2019-11-24 17:48:02.945680 | Epoch 1


Average training loss at batch  0 : 1.444
Average training loss at batch  1000 : 1.188
Average training loss at batch  2000 : 1.133
Average training loss at batch  3000 : 1.461

Average training loss after epoch  1 : 2.624
Average validation loss after epoch  1 : 1.236
2019-11-24 17:52:10.216997 | Epoch 2


Average training loss at batch  0 : 1.528
Average training loss at batch  1000 : 1.388
Average training loss at batch  2000 : 1.062
Average training loss at batch  3000 : 1.368

Average training loss after epoch  2 : 2.621
Average validation loss after epoch  2 : 1.234
2019-11-24 17:56:25.354619 | Epoch 3


Average training loss at batch  0 : 1.097
Average training loss at batch  1000 : 1.392
Average training loss at batch  2000 : 0.989
Average training loss at batch  3000 : 1.688

Average training loss after epoch  3 : 2.620
Average validation loss after epoch  3 : 1.233
2019-11-24 18:00:22.267057 | Epoch 4


...

Average training loss at batch  0 : 1.154
Average training loss at batch  1000 : 1.099
Average training loss at batch  2000 : 1.376
Average training loss at batch  3000 : 1.086

Average training loss after epoch  5 : 2.621
Average validation loss after epoch  5 : 1.233
2019-11-24 18:08:19.281430 | Epoch 6


Average training loss at batch  0 : 1.191
Average training loss at batch  1000 : 1.970
Average training loss at batch  2000 : 0.830
Average training loss at batch  3000 : 1.744

Average training loss after epoch  6 : 2.620
Average validation loss after epoch  6 : 1.233
2019-11-24 18:12:17.534749 | Epoch 7


Average training loss at batch  0 : 1.584
Average training loss at batch  1000 : 1.648
Average training loss at batch  2000 : 1.219
Average training loss at batch  3000 : 0.953

Average training loss after epoch  7 : 2.622
Average validation loss after epoch  7 : 1.233
2019-11-24 18:16:10.946037 | Epoch 8


Average training loss at batch  0 : 1.351
Average training loss at batch  1000 : 1.318
Average training loss at batch  2000 : 1.343
Average training loss at batch  3000 : 1.411

Average training loss after epoch  8 : 2.620
Average validation loss after epoch  8 : 1.233
2019-11-24 18:20:20.535310 | Epoch 9


Average training loss at batch  0 : 1.166
Average training loss at batch  1000 : 1.241
Average training loss at batch  2000 : 1.377
Average training loss at batch  3000 : 1.470

Average training loss after epoch  9 : 2.622
Average validation loss after epoch  9 : 1.233


In [42]:
torch.save(lstm_centroids, model_dir+'centroids_unlabelled')

In [ ]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server

# Evaluate Model

## Supervised Evaluation

In [14]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [16]:
## This cell will change for each model
model_folder = 'lstm_model/'

criterion = nn.CrossEntropyLoss(reduction='sum')
criterion = criterion.to(current_device)

path = os.getcwd()
model_dir = path + '/models/' + model_folder

opts = torch.load(model_dir+'opts_labelled')
model = LSTM_model(opts['embedding_matrix']) #change here depending on model
model.load_state_dict(torch.load(model_dir+'model_dict_labelled.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)

In [38]:
TP_cluster, FP_cluster=evaluation.main(model, None, val_loader, criterion, data_dir, current_device)

Total examples in val loader: 455
Assigned to cluster 1: 407
TP_rate: 0.9312039312039312
FP_rate: 0.0687960687960688
FN_rate: 0.4166666666666667
TN_rate: 0.5833333333333334


Accuracy: 0.7572686322686323
Precision: 0.9312039312039312
Recall: 0.6908704238189276
F1 score: 0.7932327548617774


## Unsupervised Evaluation

In [52]:
## This cell will change for each model
model_folder = 'lstm_model/'

criterion = KMeansCriterion(1)
criterion = criterion.to(current_device)

path = os.getcwd()
model_dir = path + '/models/' + model_folder

opts = torch.load(model_dir+'opts_unlabelled')
model = LSTM_model(opts['embedding_matrix']) #change here depending on model
model.projection = nn.Identity()
model.load_state_dict(torch.load(model_dir+'model_dict_unlabelled.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)
centroids = torch.load(model_dir+'centroids_unlabelled',map_location=lambda storage, loc: storage)

In [53]:
TP_cluster, FP_cluster=evaluation.main(model, centroids, val_loader, criterion, data_dir, current_device)

Total examples in val loader: 455
Assigned to cluster 1: 376
TP_rate: 0.9468085106382979
FP_rate: 0.05319148936170213
FN_rate: 0.5443037974683544
TN_rate: 0.45569620253164556


Accuracy: 0.7012523565849718
Precision: 0.9468085106382979
Recall: 0.6349679400343177
F1 score: 0.7601491972539055


In [48]:
#TP_cluster

In [49]:
#FP_cluster

In [50]:
#FP_cluster[FP_cluster.original == 1]